In [1]:
import os 
import librosa
import math 
import json
import matplotlib.pyplot as plt 
import numpy as np 

In [66]:
dataset_path = r'./input/notes_v2'
json_path = r'data.json'
SAMPLE_RATE = 44100

#DURATION = 1
#SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

In [160]:
def save_mfcc(dataset_path, json_path, n_mfcc=10, n_fft=2048,
             hop_length=512, num_segments=5):
    
    # Data storage dictionary
    data = {
        "mapping": [],
        "mfcc": [],
        "labels": [],
    }
    #samples_ps = int(SAMPLES_PER_TRACK/num_segments) # ps = per segment
    #expected_vects_ps = int(samples_ps/hop_length)
    
    # loop through all the genres
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # ensuring not at root
        if dirpath is not dataset_path:
            # save the semantic label
            dirpath_comp = dirpath.split("/")
            semantic_label = dirpath_comp[-1]
            data["mapping"].append(semantic_label)
            print(f"Processing: {semantic_label}")
            
            # process files for specific genre
            for f in filenames:
                # load audio file
                file_path = os.path.join(dirpath, f)
                signal,sr = librosa.load(file_path,sr=SAMPLE_RATE)
                    
                DURATION = len(signal) / sr
                SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION
                samples_ps = int(SAMPLES_PER_TRACK/num_segments) # ps = per segment
                expected_vects_ps = int(samples_ps/hop_length)
                    
                for s in range(num_segments):
                    start_sample = samples_ps * s
                    finish_sample = start_sample + samples_ps
                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                                sr = sr,
                                                n_fft = n_fft,
                                                n_mfcc = n_mfcc,
                                                hop_length = hop_length)
                    #mfcc = mfcc.T
                    # store mfcc if it has expected length 
                    data["mfcc"].append(mfcc.tolist())
                    data["labels"].append(i-1)
                    print(f"{file_path}, segment: {s+1}")

    with open(json_path,"w") as f:
        json.dump(data,f,indent=4)

In [161]:
from IPython.display import clear_output

In [162]:
save_mfcc(dataset_path, json_path)
clear_output()

In [164]:
def load_data(dataset_path):
    with open(dataset_path, 'r') as f:
        data = json.load(f)
        
    inputs = np.array(data['mfcc'])
    targets = np.array(data['labels'])
    
    return inputs, targets

In [177]:
inputs, targets = load_data(r'./data.json')

C:\Users\GODOVO~1\AppData\Local\Temp/ipykernel_10548/4225200253.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  inputs = np.array(data['mfcc'])


In [179]:
inputs = inputs.reshape(-1)

ValueError: cannot reshape array of size 3600 into shape (252,10)

In [175]:
inputs

(3600,)

In [172]:
from sklearn.model_selection import train_test_split

input_train, input_test, target_train, target_test = train_test_split(inputs, targets, test_size=0.3)
print(input_train.shape, target_train.shape)

(252, 10, 1) (252,)
